In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install keras-mdn-layer

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-jan-2021/train.csv")

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = train.drop(['id','target'],axis=1).values
y = train['target'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.4, random_state=42)

In [ ]:
from tensorflow import keras
import mdn

N_HIDDEN = 100
N_MIXES = 5

model = keras.Sequential()
model.add(keras.layers.Dense(N_HIDDEN, batch_input_shape=(None, 14), activation='relu'))
model.add(keras.layers.Dense(N_HIDDEN, activation='relu'))
model.add(mdn.MDN(1, N_MIXES))
model.compile(loss=mdn.get_mixture_loss_func(1, N_MIXES), optimizer=keras.optimizers.Adam())
model.summary()

In [ ]:
history = model.fit(x=X_train, y=y_train,epochs=50, batch_size = 3000)

In [ ]:
preds = model.predict(X_test)

In [ ]:
preds

In [ ]:
preds[:10]

In [ ]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0) 

In [ ]:
all_pi = preds[:,-N_MIXES:]

In [ ]:
all_pi = np.apply_along_axis(softmax,1,all_pi)

In [ ]:
all_mus = preds[:,0:5]

In [ ]:
k =all_pi[0]*all_mus[0]
k.sum()

In [ ]:
cv_preds = all_pi*all_mus
cv_preds = cv_preds.sum(axis=1)

should I change how the sampling is done below to get better results???

y_samples = np.apply_along_axis(mdn.sample_from_output, 1, preds, 1, N_MIXES, temp=1.0)

In [ ]:
pd.Series(cv_preds).hist(bins=200)

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
mean_squared_error(y_test, cv_preds, squared=False)

In [ ]:
test = pd.read_csv("../input/tabular-playground-series-jan-2021/test.csv").drop(['id'],axis=1)

In [ ]:
len(test)

In [ ]:
sub = pd.read_csv("../input/tabular-playground-series-jan-2021/sample_submission.csv")

In [ ]:
finalgroup = model.predict(test)

In [ ]:
all_pi_final = finalgroup[:,-N_MIXES:]

In [ ]:
all_pi_final = np.apply_along_axis(softmax,1,all_pi_final)

In [ ]:
all_mus_final = finalgroup[:,0:5]

In [ ]:
k =all_pi_final[0]*all_mus_final[0]
k.sum()

In [ ]:
final_preds = all_pi_final*all_mus_final
final_preds = final_preds.sum(axis=1)

In [ ]:
len(final_preds)

In [ ]:
pd.Series(final_preds).hist(bins=200)

intesting these are different than for cv

In [ ]:
sub['target'] = final_preds
sub

In [ ]:
sub.to_csv("submission.csv",index=False)